## Logistic Model for Binary Classification

A retail banking institution is going to float a stock trading facility for their existing customer. Since this kind of facitlity is nothing new , company knows that they will have to incetivise their customers for adopting their offerings. One way to incetiwise is to offer discounts on the commision for trading transactions.

One issue with that is that only about 10% of the customers do enought trades for earnings after discounts to be profitable. Company wants to figure out, which are those 10% customer so that it can selectively offer them discount. there is no magic way to figure that out. So company rolled out this service to about 10000+ of their customers and observed their trading behaviour for 6 months and after that they labelled them into two revenue.grids 1 and 2. using this data, now they want us to build a classification model which can be used to classify their remaining customers into these revenue grids.

In [30]:
import numpy as np
import pandas as pd

In [31]:
train_file=r'E:/Edvancer/Python/GitHub/Logistic Regression/rg_train.csv'
test_file=r'E:/Edvancer/Python/GitHub/Logistic Regression/rg_test.csv'

bd_train=pd.read_csv(train_file)
bd_test=pd.read_csv(test_file)


In [32]:
bd_test['Revenue.Grid']=np.nan
bd_train['data']='train'
bd_test['data']='test'

bd_test=bd_test[bd_train.columns]

bd_all=pd.concat([bd_train,bd_test],axis=0)

In [33]:
bd_all.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Home.Loan,Online.Purchase.Amount,Revenue.Grid,gender,region,Investment.in.Commudity,Investment.in.Equity,Investment.in.Derivative,Portfolio.Balance,data
0,2148,1,45-50,Partner,Professional,Professional,Rent Privately,">=35,000",Yes,Yes,...,2.48,0.0,2.0,Female,South West,65.87,9.27,30.93,87.48,train
1,8099,1,61-65,Partner,Retired,Retired,Own Home,"<12,500, >=10,000",No,No,...,3.99,0.0,2.0,Female,Unknown,42.46,4.49,26.23,110.73,train
2,6611,3,31-35,Partner,Professional,Professional,Own Home,">=35,000",No,No,...,0.00,0.0,2.0,Male,East Anglia,75.38,0.00,26.66,127.57,train
3,1950,Zero,55-60,Partner,Professional,Professional,Own Home,">=35,000",No,No,...,0.00,0.0,2.0,Female,North West,34.78,6.91,29.24,33.79,train
4,10857,2,51-55,Partner,Manual Worker,Manual Worker,Own Home,"<27,500, >=25,000",Yes,Yes,...,0.00,0.0,2.0,Female,South West,48.58,9.58,20.65,56.17,train


In [34]:
bd_all['Revenue.Grid'].value_counts()

#Bank want to give discount which fall in 1(10%) not 0(other 90%)

2.0    7261
1.0     863
Name: Revenue.Grid, dtype: int64

In [35]:
bd_all['Revenue.Grid']=(bd_all['Revenue.Grid']==1).astype(int)

In [36]:
bd_all['Revenue.Grid'].value_counts()

0    9292
1     863
Name: Revenue.Grid, dtype: int64

In [37]:
#How many unique value each column take??

list(zip(bd_all.columns,bd_all.dtypes,bd_all.nunique()))

[('REF_NO', dtype('int64'), 10155),
 ('children', dtype('O'), 5),
 ('age_band', dtype('O'), 13),
 ('status', dtype('O'), 5),
 ('occupation', dtype('O'), 9),
 ('occupation_partner', dtype('O'), 9),
 ('home_status', dtype('O'), 5),
 ('family_income', dtype('O'), 13),
 ('self_employed', dtype('O'), 2),
 ('self_employed_partner', dtype('O'), 2),
 ('year_last_moved', dtype('int64'), 95),
 ('TVarea', dtype('O'), 14),
 ('post_code', dtype('O'), 10040),
 ('post_area', dtype('O'), 2039),
 ('Average.Credit.Card.Transaction', dtype('float64'), 1411),
 ('Balance.Transfer', dtype('float64'), 2183),
 ('Term.Deposit', dtype('float64'), 1419),
 ('Life.Insurance', dtype('float64'), 3111),
 ('Medical.Insurance', dtype('float64'), 1589),
 ('Average.A.C.Balance', dtype('float64'), 2223),
 ('Personal.Loan', dtype('float64'), 1760),
 ('Investment.in.Mutual.Fund', dtype('float64'), 2470),
 ('Investment.Tax.Saving.Bond', dtype('float64'), 832),
 ('Home.Loan', dtype('float64'), 884),
 ('Online.Purchase.Amount'

In [38]:
# REF_NO, post_code , post_area  : drop cz are they takes too many unique values 
# children : Zero : 0 and 4+ : 4 and then convert to numeric
# age_band : dummies 
# status , occupation , occupation_partner , home_status,family_income : dummies
# self_employed, ` : dummies
# TVArea , Region , gender : dummies
# Revenue Grid : 1,2 : 1,0

In [39]:
bd_all.drop(['REF_NO','post_code','post_area'],axis=1,inplace=True)

In [40]:
bd_all["children"].value_counts()

Zero    6208
1       1848
2       1607
3        473
4+        19
Name: children, dtype: int64

In [41]:
#Convert children to numeric

bd_all['children']=np.where(bd_all['children']=='Zero',0,bd_all['children'])
bd_all['children']=np.where(bd_all['children'][:1]=='4',4,bd_all['children'])
bd_all['children']=pd.to_numeric(bd_all['children'],errors='coerce')

#if you dont add errors='coerce', it will give error as, Unable to parse string "4+" at position 916 i.e, 4+ contains 19 NA values

In [42]:
bd_all["children"].dtypes

dtype('float64')

In [43]:
bd_all["children"].value_counts()

0.0    6208
1.0    1848
2.0    1607
3.0     473
Name: children, dtype: int64

In [44]:
#Remaining Categorical Variables

cat_vars=bd_all.select_dtypes(['object']).columns
cat_vars

Index(['age_band', 'status', 'occupation', 'occupation_partner', 'home_status',
       'family_income', 'self_employed', 'self_employed_partner', 'TVarea',
       'gender', 'region', 'data'],
      dtype='object')

In [45]:
pd.get_dummies(bd_all['gender'])
#Here we can see there are 1 and 0 in male and female both
#As we know 0 for male and 1 for female
#So we dont need both column, so we will remove first column using drop_first=True

,Female,Male,Unknown
0,1,0,0
1,1,0,0
2,0,1,0
3,1,0,0
4,1,0,0
5,0,1,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [46]:
list(zip(cat_vars[:-1],bd_all.select_dtypes(['object']).nunique()))

[('age_band', 13),
 ('status', 5),
 ('occupation', 9),
 ('occupation_partner', 9),
 ('home_status', 5),
 ('family_income', 13),
 ('self_employed', 2),
 ('self_employed_partner', 2),
 ('TVarea', 14),
 ('gender', 3),
 ('region', 13)]

In [47]:
#Now we will use pd.getdummies function, but its not use any frequency cutoff 
#[:-1] is except last cz last one which is data 
#get_dummies function by default create n dummies if there are n distinct values, so we will drop fisrt value 
#once we create dummy, we concatenate bd_all and dummy columns
#then delete the original column

for col in cat_vars[:-1]:
    dummy=pd.get_dummies(bd_all[col],drop_first=True,prefix=col)
    bd_all=pd.concat([bd_all,dummy],axis=1)
    del bd_all[col]
    print(col)
del dummy

age_band
status
occupation
occupation_partner
home_status
family_income
self_employed
self_employed_partner
TVarea
gender
region


In [48]:
bd_all.shape

(10155, 96)

In [49]:
bd_all.isnull().sum()

children                           19
year_last_moved                     0
Average.Credit.Card.Transaction     0
Balance.Transfer                    0
Term.Deposit                        0
Life.Insurance                      0
Medical.Insurance                   0
Average.A.C.Balance                 0
Personal.Loan                       0
Investment.in.Mutual.Fund           0
Investment.Tax.Saving.Bond          0
Home.Loan                           0
Online.Purchase.Amount              0
Revenue.Grid                        0
Investment.in.Commudity             0
Investment.in.Equity                0
Investment.in.Derivative            0
Portfolio.Balance                   0
data                                0
age_band_22-25                      0
age_band_26-30                      0
age_band_31-35                      0
age_band_36-40                      0
age_band_41-45                      0
age_band_45-50                      0
age_band_51-55                      0
age_band_55-

In [50]:
#if number of missing value >0 then replace it with the mean
#Note:- for test, imputing value in test never make use of test data. Here we are imputing with train column only

bd_all.loc[bd_all['children'].isnull(),'children']=bd_all.loc[bd_all['data']=='train','children'].mean()


In [51]:
bd_all["children"].value_counts()

0.000000    6208
1.000000    1848
2.000000    1607
3.000000     473
0.637821      19
Name: children, dtype: int64

In [52]:
#Training data is where the data indicator column is train
#Once we are seperated, we dont need the data column any more

#From test we always drop response variable and data

bd_train=bd_all[bd_all['data']=='train']
del bd_train['data']

bd_test=bd_all[bd_all['data']=='test']
bd_test.drop(['Revenue.Grid','data'],axis=1,inplace=True)

E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [53]:
from sklearn.model_selection import train_test_split

In [54]:
bd_train1,bd_train2=train_test_split(bd_train,test_size=0.2,random_state=2)

In [55]:
#For any modelling in python, you need to pass predictor and response seperatly
#all predictor will goes in xtrain and response will goes in ytrain

#1 is axis = 1
#we are not using inplace=True cz we are not removing that column from data. 
#Here it will keep all other variables except the Intrest Rate guy


#Build model on ld_train1 using x_train1 and y_train1
x_train1=bd_train1.drop('Revenue.Grid',axis=1) 
y_train1=bd_train1['Revenue.Grid']             

#check performance on ld_train2 using x_train2 and y_train2
x_train2=bd_train2.drop('Revenue.Grid',axis=1) #Predicted/Observed value Value
y_train2=bd_train2['Revenue.Grid']  #Actual value

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


In [57]:
#alwasy use class_weight="balanced" give you a model which enable you to differentiate better between two classes if one them 
#in much more mejority
#l1 is lasso and l2 is ridge

logr=LogisticRegression(class_weight="balanced",penalty='l1')

In [58]:
logr.fit(x_train1,y_train1)

E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [59]:
prediction = logr.predict(x_train2)

#It consider internally cutoff is 0.5 which is not we are interested in
#We want to look at the probabilities, so we will use predict_proba function

In [60]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train2 ,prediction)

array([[1331,  118],
       [  27,  149]], dtype=int64)

In [61]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train2 ,prediction)

0.9107692307692308

##  predict_proba on xtrain2 i.e predicted one

In [62]:
logr.predict_proba(x_train2)

array([[0.95330957, 0.04669043],
       [0.94418569, 0.05581431],
       [0.94050537, 0.05949463],
       ...,
       [0.65580322, 0.34419678],
       [0.95242843, 0.04757157],
       [0.92781211, 0.07218789]])

In [63]:
#We want to look at which column belongs to which probabilities
#We will comes to know that first column is for 0 and second is for 1
logr.classes_

array([0, 1])

In [64]:
#Extract only second

predicted_prob=logr.predict_proba(x_train2)[:,1]
predicted_prob

array([0.04669043, 0.05581431, 0.05949463, ..., 0.34419678, 0.04757157,
       0.07218789])

## roc_auc_score on actual vs predicted_proba

In [65]:
# score model performance on the test data
roc_auc_score(bd_train2['Revenue.Grid'],predicted_prob)

0.9477068824894912

We know the tentative performance now, lets build the model on entire training to make prediction on test/production

In [66]:
#On Entire train data 

x_train=bd_train.drop('Revenue.Grid',axis=1)  #Predictor (Except response variable)
y_train=bd_train['Revenue.Grid']  #Response 

In [73]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':np.linspace(0.001,1000,10)}

#it will take 2*2*10 total 40 combinations
#So gridsearch will going to use all 40 combinations and will give you best poss parameter 

In [74]:
#When fit_intercept=True, the line of best fit is allowed to "fit" the y-axis (close to 100 in this example). 
#When fit_intercept=False, the intercept is forced to the origin (0, 0)

model=LogisticRegression(fit_intercept=True)

In [75]:
from sklearn.model_selection import GridSearchCV

In [76]:
#cv=5, but good result come when you take value between 10 to 15 but it will take more time to run

grid_search=GridSearchCV(model,param_grid=params,cv=5,scoring="roc_auc")

In [77]:
grid_search.fit(x_train,y_train)

E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linea

E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linea

E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linea

E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linea

E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linea

E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linea

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'class_weight': ['balanced', None], 'penalty': ['l1', 'l2'], 'C': array([1.00000e-03, 1.11112e+02, 2.22223e+02, 3.33334e+02, 4.44445e+02,
       5.55556e+02, 6.66667e+02, 7.77778e+02, 8.88889e+02, 1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [78]:
#We will fit logr

grid_search.best_estimator_

#C value comes to be 0.01 which is near to edge, try to expand it so you will gvet better result

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [96]:
logr=grid_search.best_estimator_

Using the report function given below you can see the cv performance of top few models as well, that will the tentative performance

In [97]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [98]:
#Tentitive Performance

report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.954 (std: 0.004)
Parameters: {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l1'}

Model with rank: 2
Mean validation score: 0.954 (std: 0.004)
Parameters: {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 3
Mean validation score: 0.948 (std: 0.009)
Parameters: {'C': 444.45, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 4
Mean validation score: 0.947 (std: 0.009)
Parameters: {'C': 111.12, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 5
Mean validation score: 0.947 (std: 0.009)
Parameters: {'C': 333.34, 'class_weight': 'balanced', 'penalty': 'l2'}



In [99]:
logr.fit(x_train,y_train)

E:\Edvancer\softwares\Python_Edvancer\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.01, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [100]:
#test_pred=logr.predict_proba(bd_test)[:,1]
#pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

In [101]:
cutoffs=np.linspace(0.01,0.99,99)
cutoffs

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [102]:
#[:,1] Extract only second column

train_score=logr.predict_proba(x_train)[:,1]

real=y_train

In [103]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
      
    KS=(TP/P)-(FP/N)
    
    
    KS_all.append(KS)

In [104]:
list(zip(cutoffs,KS_all))

#How you will find out where do i get maximum values of ks??
#for which cutoff KS is maximum

[(0.01, 0.10744699814545966),
 (0.02, 0.16305352345895308),
 (0.03, 0.21678891163333436),
 (0.04, 0.26509616687383486),
 (0.05, 0.3161910573847838),
 (0.060000000000000005, 0.375435328633122),
 (0.06999999999999999, 0.4378472076489852),
 (0.08, 0.5183006787320569),
 (0.09, 0.5520663657633449),
 (0.09999999999999999, 0.5878171338072908),
 (0.11, 0.6156369933307726),
 (0.12, 0.6366514991518841),
 (0.13, 0.6564265062813556),
 (0.14, 0.6730339056433017),
 (0.15000000000000002, 0.6890904166978523),
 (0.16, 0.6999942389722198),
 (0.17, 0.7106226170928897),
 (0.18000000000000002, 0.7184965536765809),
 (0.19, 0.7279424050423835),
 (0.2, 0.7388224491134481),
 (0.21000000000000002, 0.7481875503391745),
 (0.22, 0.7535824895395854),
 (0.23, 0.7575432360347341),
 (0.24000000000000002, 0.7640161736466333),
 (0.25, 0.7687794743995725),
 (0.26, 0.7735427751525116),
 (0.27, 0.7766771891865669),
 (0.28, 0.7808088514920344),
 (0.29000000000000004, 0.7836678213723918),
 (0.3, 0.7876047896642373),
 (0.31, 

In [105]:
#for which cutoff KS is maximum
mycutoff=cutoffs[KS_all==max(KS_all)][0]
mycutoff

0.5

In [106]:
logr.intercept_

array([0.])

In [108]:
list(zip(x_train.columns,logr.coef_[0]))

[('children', 0.0),
 ('year_last_moved', -0.001286760243802204),
 ('Average.Credit.Card.Transaction', 0.022695060231099233),
 ('Balance.Transfer', -0.005209748313563944),
 ('Term.Deposit', -0.019807442955267245),
 ('Life.Insurance', 0.014049213153280173),
 ('Medical.Insurance', -0.008534054774946037),
 ('Average.A.C.Balance', -0.002834820221026519),
 ('Personal.Loan', -0.0301654935841037),
 ('Investment.in.Mutual.Fund', 0.0008288205754218886),
 ('Investment.Tax.Saving.Bond', 0.09208837526558257),
 ('Home.Loan', -0.06891658599151126),
 ('Online.Purchase.Amount', 0.053129639139592795),
 ('Investment.in.Commudity', 0.0),
 ('Investment.in.Equity', 0.0),
 ('Investment.in.Derivative', 0.0),
 ('Portfolio.Balance', 9.730385791880658e-05),
 ('age_band_22-25', 0.0),
 ('age_band_26-30', 0.0),
 ('age_band_31-35', 0.0),
 ('age_band_36-40', 0.0),
 ('age_band_41-45', 0.0),
 ('age_band_45-50', 0.0),
 ('age_band_51-55', 0.0),
 ('age_band_55-60', 0.0),
 ('age_band_61-65', 0.0),
 ('age_band_65-70', 0.0),

## if you simply had to submit probability scores , you could do this 

In [109]:
#Predict will give hard classes
#prodict_proba will give probabilities

#Note:- you store above logr as grid_search.best_estimator_


test_score=logr.predict_proba(bd_test)[:,1]

In [ ]:
#Give a order 0 and 1
#so first column is probability of outcome being 0 and second is 1

logr.classes_

In [ ]:
pd.DataFrame(test_score).to_csv("mysubmission.csv",index=False)

## if you had to submit hardclasses , you can apply the cutoff obtained above and then submit

In [66]:
#if probability score (test_score)>cutoff then 1 otherwise 0
test_score>mycutoff

array([False, False, False, ..., False, False, False])

In [67]:
(test_score>mycutoff).sum()

404

In [111]:
#Above booleans will convert into 1 andf 0 using astype(int)

test_classes=(test_score>mycutoff).astype(int)

In [112]:
test_classes

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
pd.DataFrame(test_classes).to_csv("mysubmission.csv",index=False)